**Information:** *The weekly report supposed to be finished in week 11*

**Written by:** *Zihao Xu*

**Last updated date:** *Nov.10.2021*

#  Recap on previous work

## Existing simulations

- Set the ROS environment and test the initial simulation.
- Got familiar with the current controlling scripts.
    - Took notes of necessary message types.
    - Looked into the node graphs (subscriptions, publishers and topics).
    - Modified the existing trajectories.
- Reviewed all the ROS operations and made some notes for quick reference.
- Read through the launch file, node parameters and package information.
- Modified the launch file and package descriptions.

## Gazebo worlds
- Tested the pre-built Gazebo worlds in **Rotors** and determined some ideal environments for testing the obstacle avoidance algorithms.
- World named **warehouse**. \begin{figure}\centering\includegraphics[width=0.75\textwidth]{Figures/warehouse.png} \label{Fig1} \end{figure}
- World named **outdoor**.\begin{figure}\centering\includegraphics[width=0.75\textwidth]{Figures/outdoor.png} \label{Fig2} \end{figure}
- World named **powerplant**.\begin{figure}\centering\includegraphics[width=0.75\textwidth]{Figures/powerplant.png} \label{Fig1} \end{figure}
- World named **test_city**.\begin{figure}\centering\includegraphics[width=0.75\textwidth]{Figures/test_city.png} \label{Fig3} \end{figure}
- World named **waypoint**.\begin{figure}\centering\includegraphics[width=0.75\textwidth]{Figures/waypoint.png} \label{Fig4} \end{figure}
- It needs to be mentioned that some worlds have the **sim_time**, **real_time** and **wall_time** tags in the world definition. Sometimes this would cause the spawn immediately time out as soon as the world actually starts. Setting **sim_time** to 0 would be an easy fix, although the spawn model service is supposed to handle this case.


## Basic movements

- Dived into the source package **[Rotors](https://github.com/ethz-asl/rotors_simulator/wiki)** and checked all the existing controllers.
- Found out that the **roll_pitch_yawrate_thrust_controller** might be essential for obstacle avoidance tasks.

# Get control of basic movements

## Modify the launch file

The way to take use of the roll_pitch_yawrate_thrust_controller is similar to the way launching lee_position_controller.

In [ ]:
<node name="roll_pitch_yawrate_thrust_controller_node" pkg="rotors_control" type="roll_pitch_yawrate_thrust_controller_node" output="screen">
    <rosparam command="load" file="$(find rotors_gazebo)/resource/roll_pitch_yawrate_thrust_controller_pelican.yaml" />
    <rosparam command="load" file="$(find rotors_gazebo)/resource/pelican.yaml" />
    <remap from="odometry" to="ground_truth/odometry" />
</node>

One thing needs to pay some attention to is the type of the UAV. The files describing pre-set gains of controller and information of model must be correctly assigned. In this case, we're using the model called **pelican** according to the codes above launching the Gazebo world. This parameter is important because each model has a different set of controller gains, mass and inertia information and rotors information.

## Edit the package information

The new controller requires a publisher with the data type mav_msgs/RollPitchYawrateThrust  to the topic /command/roll_pitch_yawrate_thrust. Here is is information of the message type.

**mav_msgs/RollPitchYawrateThrust**

| Type | Name |
| :---- | :---- |
| *std_msgs/Header* | header |
| *double* | roll |
| *double* | pitch |
| *double* | yaw_rate |
| *geometry_msgs/Vector3* | thrust |

Therefore, the descriptions for the package **hunter_gazebo** should be edited and the dependency of **mav_msgs** should be added. In this case, both the **CMakeLists.txt** and **package.xml** are modified.

## Test the new controller

To test how the controller work, a new launch file initiates the hunter drone with the additionally controller was created. The script assigning the trajectories is also edited to test the new controller. 

That is to say, we can have a combined controller which can either automatically follow the assigned trajectories, or move according to direct and simple commands.

# Check the vision information

## Existing cameras

There are three depth cameras on the existing drone model named hunter. Two of them are placed forward like human eyes and the last one is placed downward. The names are as follows.

- **camera_downward**
- **left**
- **right**

## Available messages

According to the topic lists, some of the available topics of each camera are listed below.

### image_raw

- Type: **sensor_msgs/Image**
- The message contains an uncompressed image.
- (0,0) is at the top-left corner of the image.

| Type | Name | Note |
| :---- | :---- | :---- |
| *std_msgs/Header* | header | |
| *uint32* | height | Number of rows |
| *uint32* | width | Number of columns |
| *string* | encoding | Encoding of pixels |
| *uint8* | is_bigendian | If the data is [big endian](https://en.wikipedia.org/wiki/Endianness) |
| *uint32* | step | Full row length in bytes |
| *uint8[]* | data | Actual matrix data |

- In the simulation, some important parameters are
    - height: 480
    - width: 752
    - string: mono8

### image_raw/compressed

- Type: **sensor_msgs/CompressedImage**
- The message contains a compressed image.

| Type | Name | Note |
| :---- | :---- | :---- |
| *std_msgs/Header* | header | |
| *string* | format | Format of the data such as *png* and *jpeg* |
| *uint8[]* | data | Actual matrix data |

- In this simulation, the image is compressed to the format *jpeg*.

###  image_raw/compresssedDepth

- Type: **sensor_msgs/CompressedImage**
- In this simulation, the compression failed because the input format is *mono8* while the compression process requires single-channel 32bit-floating point or 16bit raw depth images.
- If we would like to make use of the depth information, we may need to modify the cameras so that we can get the depth information.

## Examples

Here is the currently available images from the cameras. The following images were taken in the **test_city** world while no movements is assigned to the drone.

- The situation of the drone. \begin{figure}\centering\includegraphics[width=0.75\textwidth]{Figures/Situation.png} \label{Fig5} \end{figure}
- The output of the left camera. \begin{figure}\centering\includegraphics[width=0.75\textwidth]{Figures/Left_camera.png} \label{Fig6} \end{figure}
- The output of the right camera. \begin{figure}\centering\includegraphics[width=0.75\textwidth]{Figures/right_camera.png} \label{Fig7} \end{figure}

## Possible works

- The motor itself occurred in the images. Modify the camera positions when needed.
- The current images are gray. Not sure if color images are needed.
- The depth information requires modification of the type of outputs from the cameras.

# Existing collision avoidance systems and approaches

Given that I've got control of the basic movements by changing the controller and got access to the camera information, I searched for literatures in the area of collision avoidance and tried to find out any well-build vision-based systems so that I can conveniently implement a stable algorithms to the simulation. 

I mainly focused on a paper which claims to provide a comprehensive review of collision avoidance strategies used for unmanned vehicles, with the main emphasis on unmanned aerial vehicles (UAV). The paper is published in [IEEE Access](https://ieeexplore.ieee.org/document/9108245) on Jun. 04, 2020.

Right now I would like to try to implement the vision-based algorithms as a start therefore I only focused on the camera related to cameras. The paper mentions an approach based on stereo cameras which can obtain the absolute depth using the intrinsic and extrinsic parameters of the cameras. The approach is successfully implemented on a 6DoF drone with a fuzzy controller to smooth the response of the controller. I would like to look into those references and check if the perception and control algorithms can be applied to this project.

# Modified files

Here is the list of files I've modified and plan to commit and upload to the git repo.

**Edited**:

- *hunter_gazebo/CMkaeLists.txt*
    - Modified the package dependencies.
- *hunter_gazebo/package.xml*
    - Modified the package dependencies.
- *hunter_gazebo/launch/run_simulation.launch*
    - Added detailed comments and deleted non-necessary nodes.
- *hunter_gazebo/scripts/follow_waypoints.py*
    - Added detailed comments, modified the assigned trajectories.
- *rotors_gazebo/worlds/test_city.world*
    - Modified the *sim_time* parameter for successful implementation.
    
**Added**:

- *hunter_gazebo/launch/obstacle_avoidance.launch*
    - A new launch file which initiate a hunter drone with two controllers in the **test_city** world.
    - Will be used for further obstacle avoidance tasks.
- *hunter_gazebo/scripts/cas_controller.py*
    - A test script using both the new controller and the old controller.
    - Add subscribers for vision informations.
    - Will be developed for further obstacle avoidance tasks.

# Summary

In the past two weeks, I 

- debugged the new controller nodes with new scripts and tested its performance along with the lee position controller.
- carefully checked the available vision information since I proposed to use the depth information for obstacle detection.
- searched for and read papers related to the area of collision avoidance and tried to find possible algorithms which could be applied to this project.

In the next two week, I plan to

- modify the camera settings so that the depth information could be obtained.
- dive deeper into the application of stereo cameras introduced in the paper mentioned above and see if there is existing algorithms for convenient implementation.

Please let me know if anything I'm doing is not on the right track. Any advice of suggestions would be greatly appreciated.